In [106]:
import requests
import os
import json
import pandas as pd
from datetime import datetime, timedelta
import time
import duckdb

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

API_KEY = os.environ.get('NYTIMES_API_KEY')
API_SECRET = os.environ.get('NYTIMES_API_SECRET')

In [ ]:
headers = {
  "Accept": "application/json"
}

In [69]:
best_sellers = []

In [70]:
best_sellers_url = f"https://api.nytimes.com/svc/books/v3/lists/full-overview.json?published_date=2021-01-01&api-key={API_KEY}"
response_best_sellers = requests.get(best_sellers_url, headers=headers).text
best_sellers.append(json.loads(response_best_sellers))
next_published_date = json.loads(response_best_sellers)['results']['next_published_date']

In [71]:
next_published_date

'2021-01-10'

In [75]:
while '2024' not in next_published_date:
    best_sellers_url = f"https://api.nytimes.com/svc/books/v3/lists/full-overview.json?published_date={next_published_date}&api-key={API_KEY}"
    response_best_sellers = requests.get(best_sellers_url, headers=headers).text
    next_published_date = json.loads(response_best_sellers)['results']['next_published_date']
    best_sellers.append(json.loads(response_best_sellers))
    time.sleep(15)

In [90]:
df = pd.DataFrame()

In [91]:
for best_seller in best_sellers:
    for lst in best_seller['results']['lists']:
        df_temp = pd.DataFrame(lst['books'])
        df_temp['list_name'] = lst['list_name']
        df_temp['list_name_encoded'] = lst['list_name_encoded']
        df_temp['display_name'] = lst['display_name']
        df_temp['updated'] = lst['updated']
        df_temp['published_date'] = best_seller['results']['published_date']
        df_temp['bestsellers_date'] = best_seller['results']['bestsellers_date']
        df_temp['published_date_description'] = best_seller['results']['published_date_description']
        df_temp['previous_published_date'] = best_seller['results']['previous_published_date']
        df_temp['next_published_date'] = best_seller['results']['next_published_date']

        df = pd.concat([df, df_temp])

In [96]:
df['created_date'] = pd.to_datetime(df['created_date'])
df['updated_date'] = pd.to_datetime(df['updated_date'])
df['published_date'] = pd.to_datetime(df['published_date'])
df['bestsellers_date'] = pd.to_datetime(df['bestsellers_date'])
df['previous_published_date'] = pd.to_datetime(df['previous_published_date'])
df['next_published_date'] = pd.to_datetime(df['next_published_date'])


In [98]:
df.to_parquet('nyt_bestsellers.parquet')

In [107]:
df

,age_group,amazon_product_url,article_chapter_link,author,book_image,book_image_width,book_image_height,book_review_link,book_uri,btrn,contributor,contributor_note,created_date,description,first_chapter_link,price,primary_isbn10,primary_isbn13,publisher,rank,rank_last_week,sunday_review_link,title,updated_date,weeks_on_list,buy_links,list_name,list_name_encoded,display_name,updated,published_date,bestsellers_date,published_date_description,previous_published_date,next_published_date
0,,https://www.amazon.com/dp/0385545967?tag=thene...,,John Grisham,https://storage.googleapis.com/du-prd/books/im...,329.0,500.0,,nyt://book/33a48cf6-d7f3-5113-aa1e-6adcbb3853c3,220231703,by John Grisham,,2020-12-23 23:02:09,The third book in the Jake Brigance series. A ...,,0.00,0385545967,9780385545969,Doubleday,1,2,,A TIME FOR MERCY,2020-12-23 23:02:09,10,"[{'name': 'Amazon', 'url': 'https://www.amazon...",Combined Print and E-Book Fiction,combined-print-and-e-book-fiction,Combined Print & E-Book Fiction,WEEKLY,2021-01-03,2020-12-19,,2020-12-27,2021-01-10
1,,https://www.amazon.com/dp/0316420255?tag=thene...,,James Patterson,https://storage.googleapis.com/du-prd/books/im...,322.0,500.0,,nyt://book/662286db-c00a-50de-9ba0-dec968f24b50,220306544,by James Patterson,,2020-12-23 23:02:09,The 28th book in the Alex Cross series. An inv...,,0.00,0316420255,9780316420259,"Little, Brown",2,3,,DEADLY CROSS,2020-12-23 23:02:09,4,"[{'name': 'Amazon', 'url': 'https://www.amazon...",Combined Print and E-Book Fiction,combined-print-and-e-book-fiction,Combined Print & E-Book Fiction,WEEKLY,2021-01-03,2020-12-19,,2020-12-27,2021-01-10
2,,https://www.amazon.com/dp/1524761338?tag=thene...,,Ernest Cline,https://storage.googleapis.com/du-prd/books/im...,328.0,500.0,,nyt://book/473f18d4-0433-5c42-abaa-54c7c9dd26e5,220264849,by Ernest Cline,,2020-12-23 23:02:09,"In a sequel to “Ready Player One,” Wade Watts ...",,0.00,1524761338,9781524761332,Ballantine,3,1,,READY PLAYER TWO,2020-12-23 23:02:09,4,"[{'name': 'Amazon', 'url': 'https://www.amazon...",Combined Print and E-Book Fiction,combined-print-and-e-book-fiction,Combined Print & E-Book Fiction,WEEKLY,2021-01-03,2020-12-19,,2020-12-27,2021-01-10
3,,https://www.amazon.com/dp/0525536299?tag=thene...,,Brit Bennett,https://storage.googleapis.com/du-prd/books/im...,331.0,500.0,,nyt://book/6db17f1d-4bd0-562a-80ef-efa2a272cf64,220197315,by Brit Bennett,,2020-12-23 23:02:09,The lives of twin sisters who run away from a ...,,0.00,0525536299,9780525536291,Riverhead,4,4,,THE VANISHING HALF,2020-12-23 23:02:09,29,"[{'name': 'Amazon', 'url': 'https://www.amazon...",Combined Print and E-Book Fiction,combined-print-and-e-book-fiction,Combined Print & E-Book Fiction,WEEKLY,2021-01-03,2020-12-19,,2020-12-27,2021-01-10
4,,https://www.amazon.com/dp/1538728575?tag=thene...,,Nicholas Sparks,https://storage.googleapis.com/du-prd/books/im...,329.0,500.0,,nyt://book/b9bf792c-a853-54ce-8f33-7117f51be365,220272339,by Nicholas Sparks,,2020-12-23 23:02:09,A doctor serving in the Navy in Afghanistan go...,,0.00,1538728575,9781538728574,Grand Central,5,5,,THE RETURN,2020-12-23 23:02:09,12,"[{'name': 'Amazon', 'url': 'https://www.amazon...",Combined Print and E-Book Fiction,combined-print-and-e-book-fiction,Combined Print & E-Book Fiction,WEEKLY,2021-01-03,2020-12-19,,2020-12-27,2021-01-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,,https://www.amazon.com/dp/0593481518?tag=thene...,,Natasha Preston,https://storage.googleapis.com/du-prd/books/im...,333.0,500.0,,nyt://book/f3696062-9e49-56ab-88f5-b36677dc46fa,224571181,by Natasha Preston,,2023-12-07 00:40:21,,,0.00,0593481518,9780593481516,Delacorte,6,0,,THE HAUNTING,2023-12-07 00:40:21,0,"[{'name': 'Amazon', 'url': 'https://www.amazon...",Young Adult Paperback Monthly,young-adult-paperback-monthly,Young Adult Paperback,MONTHLY,2023-12-31,2023-12-16,,2023-12-24,2024-01-07
6,,https://www.amazon.com/dp

In [108]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36110 entries, 0 to 9
Data columns (total 35 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   age_group                   36110 non-null  object        
 1   amazon_product_url          36110 non-null  object        
 2   article_chapter_link        36110 non-null  object        
 3   author                      36110 non-null  object        
 4   book_image                  36105 non-null  object        
 5   book_image_width            36105 non-null  float64       
 6   book_image_height           36105 non-null  float64       
 7   book_review_link            36110 non-null  object        
 8   book_uri                    36110 non-null  object        
 9   btrn                        36110 non-null  object        
 10  contributor                 36110 non-null  object        
 11  contributor_note            36110 non-null  object        
 12 

In [103]:
with duckdb.connect("nyt.db") as con:
    con.sql("CREATE TABLE bestsellers AS SELECT * FROM 'nyt_bestsellers.parquet'")
    con.table("bestsellers").show()

┌───────────────┬──────────────────────┬──────────────────────┬───┬─────────────────────┬───────────────────┐
│   age_group   │  amazon_product_url  │ article_chapter_link │ … │ next_published_date │ __index_level_0__ │
│    varchar    │       varchar        │       varchar        │   │      timestamp      │       int64       │
├───────────────┼──────────────────────┼──────────────────────┼───┼─────────────────────┼───────────────────┤
│               │ https://www.amazon…  │                      │ … │ 2021-01-10 00:00:00 │                 0 │
│               │ https://www.amazon…  │                      │ … │ 2021-01-10 00:00:00 │                 1 │
│               │ https://www.amazon…  │                      │ … │ 2021-01-10 00:00:00 │                 2 │
│               │ https://www.amazon…  │                      │ … │ 2021-01-10 00:00:00 │                 3 │
│               │ https://www.amazon…  │                      │ … │ 2021-01-10 00:00:00 │                 4 │
│         

In [134]:
with duckdb.connect("nyt.db") as con:
    con.sql("""
                SELECT
                    title
                    ,count(*) as weeks_as_top_3
                FROM bestsellers
                where year(published_date) = 2022
                and rank <= 3
                group by title
                order by 2 desc""").show()
    con.sql("""COPY(
                    SELECT
                        title
                        ,count(*) as weeks_as_top_3
                    FROM bestsellers
                    where year(published_date) = 2022
                    and rank <= 3
                    group by title
                    order by 2 desc
                ) TO 'top3.csv' (HEADER, DELIMITER ',')""")

┌───────────────────────────┬────────────────┐
│           title           │ weeks_as_top_3 │
│          varchar          │     int64      │
├───────────────────────────┼────────────────┤
│ IT ENDS WITH US           │            106 │
│ ATOMIC HABITS             │            103 │
│ WHERE THE CRAWDADS SING   │             91 │
│ THE BODY KEEPS THE SCORE  │             89 │
│ VERITY                    │             69 │
│ THEY BOTH DIE AT THE END  │             52 │
│ DIARY OF A WIMPY KID      │             50 │
│ I'M GLAD MY MOM DIED      │             49 │
│ THE ONE AND ONLY IVAN     │             47 │
│ GREENLIGHTS               │             44 │
│      ·                    │              · │
│      ·                    │              · │
│      ·                    │              · │
│ WHY NOT YOU?              │              1 │
│ TROUBLEMAKER              │              1 │
│ RED NOTICE                │              1 │
│ THE FIBER FUELED COOKBOOK │              1 │
│ GROUND ZERO

In [135]:
with duckdb.connect("nyt.db") as con:
    con.sql("""
                SELECT
                    list_name
                    ,COUNT(DISTINCT title) as unique_titles
                FROM bestsellers
                GROUP BY list_name
                ORDER BY 2 ASC
                """).show()
    con.sql("""COPY(
                    SELECT
                        list_name
                        ,COUNT(DISTINCT title) as unique_titles
                    FROM bestsellers
                    GROUP BY list_name
                    ORDER BY 2 ASC
                ) TO least_unique.csv (HEADER, DELIMITER ',')
                """)

┌──────────────────────────────────────┬───────────────┐
│              list_name               │ unique_titles │
│               varchar                │     int64     │
├──────────────────────────────────────┼───────────────┤
│ Middle Grade Paperback Monthly       │            53 │
│ Young Adult Paperback Monthly        │            68 │
│ Series Books                         │            77 │
│ Business Books                       │            96 │
│ Childrens Middle Grade Hardcover     │           123 │
│ Paperback Nonfiction                 │           170 │
│ Audio Nonfiction                     │           172 │
│ Young Adult Hardcover                │           180 │
│ Graphic Books and Manga              │           186 │
│ Picture Books                        │           196 │
│ Audio Fiction                        │           207 │
│ Trade Fiction Paperback              │           231 │
│ Advice How-To and Miscellaneous      │           286 │
│ Mass Market Monthly          

In [136]:
with duckdb.connect("nyt.db") as con:
    con.sql("""with points as(
                SELECT
                    publisher
                    ,CASE
                        WHEN rank = 1 THEN 5
                        WHEN rank = 2 THEN 4
                        WHEN rank = 3 THEN 3
                        WHEN rank = 4 THEN 2
                        WHEN rank = 5 THEN 1
                        ELSE 0
                    END AS points
                    ,published_date
                FROM bestsellers
                ),
                quarter as(
                    SELECT
                        publisher
                        ,SUM(points) as points
                        ,DATE_TRUNC('quarter', published_date) as quarter_start
                        FROM points
                        GROUP BY publisher, quarter_start
                )
                SELECT
                    *
                    FROM quarter
                    ORDER BY quarter_start, points DESC                    
                """).show()
    con.sql("""COPY(
                    with points as(
                        SELECT
                            publisher
                            ,CASE
                                WHEN rank = 1 THEN 5
                                WHEN rank = 2 THEN 4
                                WHEN rank = 3 THEN 3
                                WHEN rank = 4 THEN 2
                                WHEN rank = 5 THEN 1
                                ELSE 0
                            END AS points
                            ,published_date
                        FROM bestsellers
                        ),
                        quarter as(
                            SELECT
                                publisher
                                ,SUM(points) as points
                                ,DATE_TRUNC('quarter', published_date) as quarter_start
                                FROM points
                                GROUP BY publisher, quarter_start
                        )
                        SELECT
                            *
                            FROM quarter
                            ORDER BY quarter_start, points DESC
                ) TO publisher_rank.csv (HEADER, DELIMITER ',')                
                """)

┌──────────────────────┬────────┬───────────────┐
│      publisher       │ points │ quarter_start │
│       varchar        │ int128 │     date      │
├──────────────────────┼────────┼───────────────┤
│ Scholastic           │    309 │ 2021-01-01    │
│ Random House Audio   │    193 │ 2021-01-01    │
│ Crown                │    158 │ 2021-01-01    │
│ HarperCollins        │    145 │ 2021-01-01    │
│ Avon                 │    145 │ 2021-01-01    │
│ Little, Brown        │    134 │ 2021-01-01    │
│ Amulet               │     94 │ 2021-01-01    │
│ Avery                │     93 │ 2021-01-01    │
│ St. Martin's         │     93 │ 2021-01-01    │
│ Ember                │     89 │ 2021-01-01    │
│   ·                  │      · │     ·         │
│   ·                  │      · │     ·         │
│   ·                  │      · │     ·         │
│ Atria/One Signal     │      0 │ 2023-10-01    │
│ IDW                  │      0 │ 2023-10-01    │
│ Silhouette           │      0 │ 2023-10-01    │


In [137]:
with duckdb.connect("nyt.db") as con:
    con.sql("""with teams as(
                SELECT
                    title
                    ,publisher
                    ,list_name
                    ,CASE
                        WHEN rank = 1 THEN 'Team Jake'
                        WHEN rank = 3 THEN 'Team Pete'
                        ELSE NULL
                    END AS teams
                    ,published_date
                FROM bestsellers
                )
                SELECT
                    *
                    FROM teams
                    WHERE teams IS NOT NULL
                        AND year(published_date) = 2023        
                """).show()
    
    con.sql("""COPY(
                    with teams as(
                    SELECT
                        title
                        ,publisher
                        ,list_name
                        ,CASE
                            WHEN rank = 1 THEN 'Team Jake'
                            WHEN rank = 3 THEN 'Team Pete'
                            ELSE NULL
                        END AS teams
                        ,published_date
                    FROM bestsellers
                    )
                    SELECT
                        *
                        FROM teams
                        WHERE teams IS NOT NULL
                            AND year(published_date) = 2023
                ) TO pete_jake_team.csv (HEADER, DELIMITER ',')        
                """)

┌─────────────────────────────┬───────────────┬──────────────────────────────────────┬───────────┬─────────────────────┐
│            title            │   publisher   │              list_name               │   teams   │   published_date    │
│           varchar           │    varchar    │               varchar                │  varchar  │      timestamp      │
├─────────────────────────────┼───────────────┼──────────────────────────────────────┼───────────┼─────────────────────┤
│ IT STARTS WITH US           │ Atria         │ Combined Print and E-Book Fiction    │ Team Jake │ 2023-01-01 00:00:00 │
│ LESSONS IN CHEMISTRY        │ Doubleday     │ Combined Print and E-Book Fiction    │ Team Pete │ 2023-01-01 00:00:00 │
│ THE LIGHT WE CARRY          │ Crown         │ Combined Print and E-Book Nonfiction │ Team Jake │ 2023-01-01 00:00:00 │
│ FRIENDS, LOVERS, AND THE …  │ Flatiron      │ Combined Print and E-Book Nonfiction │ Team Pete │ 2023-01-01 00:00:00 │
│ LESSONS IN CHEMISTRY        │ 